In [3]:
# 📚 Imports
from datasets import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pandas as pd

# 🔄 Load both splits
dataset = load_dataset("Suvo696/InSDNN")
df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()

print("Train shape:", df_train.shape)
print("Test shape:", df_test.shape)


Train shape: (275110, 84)
Test shape: (68778, 84)


In [6]:
df_train

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,162175,200.175.2.130,43226,192.168.3.130,80,6,9/1/2020 16:30,15163,4,4,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PROBE
1,62805,192.168.20.133,51325,192.168.20.2,53,17,5/2/2020 13:35,3225,1,3,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NORMAL
2,176020,28.240.238.46,0,192.168.3.130,0,0,10/1/2020 5:56,44,0,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDOS
3,119202,255.79.185.131,0,192.168.20.134,0,0,4/2/2020 15:20,17,0,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDOS
4,80743,192.168.20.133,49140,74.125.193.157,443,6,5/2/2020 13:43,29567478,4,8,...,0,16078.0,0.0,16078.0,16078.0,29500000.0,0.0,29500000.0,29500000.0,NORMAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275105,61870,192.168.20.133,49709,192.168.20.2,53,17,5/2/2020 12:07,3287,1,3,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NORMAL
275106,151697,200.175.2.130,43869,192.168.3.130,6378,6,9/1/2020 17:32,11,0,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PROBE
275107,122318,39.8.159.32,0,192.168.20.134,0,0,4/2/2020 15:20,22,0,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDOS
275108,162850,192.168.3.130,80,200.175.2.130,43989,6,9/1/2020 16:30,62399408,3,7,...,0,1317.0,0.0,1317.0,1317.0,62400000.0,0.0,62400000.0,62400000.0,PROBE


In [4]:
from datasets import load_dataset

# Load dataset from Hugging Face
dataset = load_dataset("Suvo696/InSDNN", data_files={"train": "train.csv"}, split="train")
dataset = dataset.shuffle(seed=42)  # adjust as needed

# Get all column names
all_columns = dataset.column_names

# Create natural-language multiline text using all fields
def format_all_columns_multiline(row):
    lines = [f"{col} is {row[col]}" for col in all_columns if col != "Label"]
    lines.append(f"Label is {'Attack' if row['Label'].lower() != 'normal' else 'Normal'}")
    return "\n".join(lines)

# Apply and preserve all data + add 'label'
def add_text_and_label(example):
    return {
        **example,
        "text": format_all_columns_multiline(example),
        "label": 0 if example["Label"].lower() == "normal" else 1
    }

# Final mapped dataset
dataset = dataset.map(add_text_and_label)


Map: 100%|██████████| 275110/275110 [03:53<00:00, 1178.31 examples/s]


In [6]:
print(dataset[0])

{'Flow ID': '192.168.20.134-43.105.89.211-0-0-0', 'Src IP': '43.105.89.211', 'Src Port': 0, 'Dst IP': '192.168.20.134', 'Dst Port': 0, 'Protocol': 0, 'Timestamp': '4/2/2020 15:20', 'Flow Duration': 2, 'Tot Fwd Pkts': 0, 'Tot Bwd Pkts': 2, 'TotLen Fwd Pkts': 0.0, 'TotLen Bwd Pkts': 0.0, 'Fwd Pkt Len Max': 0, 'Fwd Pkt Len Min': 0, 'Fwd Pkt Len Mean': 0.0, 'Fwd Pkt Len Std': 0.0, 'Bwd Pkt Len Max': 0, 'Bwd Pkt Len Min': 0, 'Bwd Pkt Len Mean': 0.0, 'Bwd Pkt Len Std': 0.0, 'Flow Byts/s': 0.0, 'Flow Pkts/s': 1000000.0, 'Flow IAT Mean': 2.0, 'Flow IAT Std': 0.0, 'Flow IAT Max': 2.0, 'Flow IAT Min': 2.0, 'Fwd IAT Tot': 0.0, 'Fwd IAT Mean': 0.0, 'Fwd IAT Std': 0.0, 'Fwd IAT Max': 0.0, 'Fwd IAT Min': 0.0, 'Bwd IAT Tot': 2.0, 'Bwd IAT Mean': 2.0, 'Bwd IAT Std': 0.0, 'Bwd IAT Max': 2.0, 'Bwd IAT Min': 2.0, 'Fwd PSH Flags': 0, 'Bwd PSH Flags': 0, 'Fwd URG Flags': 0, 'Bwd URG Flags': 0, 'Fwd Header Len': 0, 'Bwd Header Len': 0, 'Fwd Pkts/s': 0.0, 'Bwd Pkts/s': 1000000.0, 'Pkt Len Min': 0, 'Pkt Len M

In [ ]:
import torch
import pandas as pd
import json
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer,
    BitsAndBytesConfig
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# === Tokenizer and model setup ===
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# === Tokenization ===
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

def tokenize_label(example):
    return tokenizer(example["label_str"], truncation=True, padding="max_length", max_length=10)

tokenized_inputs = dataset.map(tokenize)
tokenized_labels = dataset.map(tokenize_label)

# === Combine input and labels ===
tokenized_dataset = tokenized_inputs.map(
    lambda e, l=tokenized_labels: {"labels": l["input_ids"]},
    remove_columns=["__index_level_0__"] if "__index_level_0__" in tokenized_inputs.column_names else []
)

# === Quantization config ===
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# === Load model in 4-bit ===
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# === Apply LoRA ===
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# === Training Arguments ===
training_args = TrainingArguments(
    output_dir="./mistral_sdn_lora_run1",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    bf16=torch.cuda.is_bf16_supported(),
    report_to="none",
    save_total_limit=2,
    logging_dir="./mistral_sdn_lora_run1/logs",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

# === Trainer ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset.select(range(4500)),
    eval_dataset=tokenized_dataset.select(range(4500, 5000)),
    tokenizer=tokenizer
)

# === Train and Save Everything ===
train_result = trainer.train()

# Save final model and tokenizer
trainer.save_model("./mistral_sdn_lora_run1/final_model")
tokenizer.save_pretrained("./mistral_sdn_lora_run1/final_model")

# Save evaluation metrics
metrics = trainer.evaluate()
with open("./mistral_sdn_lora_run1/eval_results.json", "w") as f:
    json.dump(metrics, f, indent=2)

# Save full training log
log_history = trainer.state.log_history
pd.DataFrame(log_history).to_csv("./mistral_sdn_lora_run1/training_log.csv", index=False)

print("✅ Training complete. All artifacts saved.")
